## Executive summary

| | |
| --- | --- |
| Problem | GCNs suffer from training difficulty due to non-linear activations, and over-smoothing problem. |
| Hypothesis | removing non-linearities would enhance recommendation performance.  |
| Solution | Linear model with residual network structure |
| Dataset | Gowalla, Amazon-books |
| Preprocessing | we remove users (items) that have less than 10 interaction records. After that, we randomly select 80% of the records for training, 10% for validation and the remaining 10% for test. |
| Metrics | HR, NDCG |
| Hyperparams | There are two important parameters: the dimension D of the user and item embedding matrix E, and the regularization parameter λ in the objective function. The embedding size is fixed to 64. We try the regularization parameter λ in the range [0.0001, 0.001, 0.01, 0.1], and find λ = 0.01 reaches the best performance. |
| Models | LR-GCCF |
| Cluster | PyTorch with GPU |

## Model

![Figure: The overall architecture of LR-GCCF](https://github.com/RecoHut-Stanzas/S794944/raw/main/images/Overall_framework.jpg)

Figure: The overall architecture of LR-GCCF

## Graph Convolutional Networks (GCN)

GCN is a representative model of graph neural networks that applies message passing to aggregate neighborhood information. GCN-based methods can effectively learn the behavioral patterns between users and items by directly capturing the collaborative signals inherent in the user-item interactions. Typical GCN-based methods include GC-MC, PinSage, and NGCF.

GCN-based methods model a set of user-item interactions as a user-item bipartite graph and then perform the following three steps:

1. **Initialization** - They randomly set the initial embedding $𝒆^0$ of all user 𝑢 and item 𝑣, denoted as $𝒆_u^0,𝒆_𝑣^0 \in \mathbb{R}^𝐷$, where 𝐷 denotes the embedding size.
2. **Propagation** - First of all, this propagation step is iterated 𝐾 times, i.e., 𝐾 layers of embedding propagation. Given the 𝐾 layers, the embedding of a user node 𝑢 (resp. an item node 𝑣) in 𝑖-th layer is updated based on the embeddings of 𝑢’s (resp. 𝑣’s) neighbors $𝑁_𝑢$ (resp. $𝑁_𝑣$) in (𝑖 − 1)-th layer as $e_u^i = \sigma(\sum_{v \in N_u}e_v^{i-1}W_i)$, and $e_v^i = \sigma(\sum_{u \in N_v}e_u^{i-1}W_i)$, where 𝜎 denotes a non-linear activation function, e.g., ReLU, and $𝑾_𝑖 \in \mathbb{R}^{𝐷×𝐷}$ is a trainable transformation matrix. There exist some other variations: i) including the self-embeddings, i.e., $𝑁_𝑢 = 𝑁_𝑢 ∪ \{𝑢\}$ and $𝑁_𝑣 = 𝑁_𝑣 ∪ \{𝑣\}$, ii) removing the transformation matrix, and iii) removing the non-linear activation, which is in particular called as linear propagation.
3. **Prediction** - The preference of user 𝑢 to item 𝑣 is typically predicted using the dot product between the user 𝑢’s and item 𝑣’s embeddings in the last layer 𝐾, i.e., $𝒆_𝑢^𝐾$ and $𝒆_𝑣^𝐾$, as $\hat{r}_{u,v}=e_u^K \odot e_v^K$.

### Message Passing Procedure

[https://youtu.be/ijmxpItkRjc](https://youtu.be/ijmxpItkRjc)

The message passing layer with self-loops is defined as follows:

$$E^{(l+1)} = \sigma(\hat{D}^{-1/2} \hat{A}\hat{D}^{-1/2}E^{(l)}W^{(l)})$$

where,

- $\hat{A} = A + I$, and $\hat{D} = D + I$
- $𝐴$, $𝐷$, $𝐼$ are the adjacency matrix, the diagonal node degree matrix, and the identity matrix, respectively
- $𝐼$ is used to integrate self-loop connections on nodes
- $𝐸^{(𝑙)}$ and $W^{(𝑙)}$ denote the representation matrix and the weight matrix for the $𝑙$-th layer
- $\sigma(·)$ is a non-linear activation function (e.g., ReLU).

LightGCN is the simplified GCN model that removes feature transformations (i.e., $𝑊^{(𝑙)}$) and non-linear activations (i.e., 𝜎). Its message passing layer can thus be expressed as follows:

$$E^{(l+1)} = (\hat{D}^{-1/2} \hat{A}\hat{D}^{-1/2}E^{(l)})$$

Given self-loop connections, we can rewrite the message passing operations for user 𝑢 and item 𝑖 as follows:

$$e_u^{(l+1)} = \dfrac{1}{d_u + 1} e_u^{(l)} + \sum_{k \in \mathcal{N}(u)} \dfrac{1}{\sqrt{d_u+1}\sqrt{d_k+1}} e_k^{(l)}$$

$$e_i^{(l+1)} = \dfrac{1}{d_i + 1} e_i^{(l)} + \sum_{v \in \mathcal{N}(i)} \dfrac{1}{\sqrt{d_i+1}\sqrt{d_i+1}} e_v^{(l)}$$

where,

- 𝑢 and 𝑣 denote users while 𝑖 and 𝑘 denote items
- $e_u^{(l)}$ and $e_i^{(l)}$ denote the embeddings of user 𝑢 and item 𝑖 at layer 𝑙
- $\mathcal{N}(𝑢)$ and $\mathcal{N}(𝑖)$ represent their neighbor node sets, respectively
- $𝑑_𝑢$ denotes the original degree of the node 𝑢.

LightGCN takes the dot product of the two embedding as the final logit to capture the preference of user 𝑢 on item 𝑖. Thus we obtain:

$$e_u^{(l+1)} \cdot e_i^{(l+1)} = \alpha_{ui}(e_u^{(l)} \cdot e_i^{(l)}) + \sum_{k \in \mathcal{N}(u)}\alpha_{ik}(e_i^{(l)} \cdot e_k^{(l)}) + \sum_{v \in \mathcal{N}(i)}\alpha_{uv}(e_u^{(l)} \cdot e_v^{(l)}) + \sum_{k \in \mathcal{N}(u)}\sum_{v \in \mathcal{N}(i)}\alpha_{kv}(e_k^{(l)} \cdot e_v^{(l)})$$

Therefore, we can observe that multiple different types of collaborative signals, including user-item relationships (𝑢-𝑖 and 𝑘-𝑣), item-item relationships (𝑘-𝑖), and user-user relationships (𝑢-𝑣), are captured when training GCN-based models with message passing layers. This also reveals why GCN-based models are effective for CF.

## Over-smoothing Problem in GCNs

Over-smoothing is caused as they use only the embeddings updated through the last layer in the prediction layer. Specifically, as the number of layers increases, the embedding of a node will be influenced more from its neighbors’ embeddings. As a result, the embedding of a node in the last layer becomes similar to the embeddings of many directly/indirectly connected nodes. This phenomenon prevents most of the existing GCN-based methods from effectively utilizing the information of high-order neighborhood. Empirically, this is also shown by the fact that most of non-linear GCN-based methods show better performance when using only a few layers instead of deep networks. The common solution is **Residual Prediction** - Utilize the embeddings from all layers for prediction. After that, perform residual prediction, which predict each user’s preference to each item with the multiple embeddings from the multiple layers.

## Tutorials

### LR-GCCF Model Training on Gowalla Dataset in PyTorch

[Link to notebook →](https://github.com/RecoHut-Stanzas/S794944/blob/main/nbs/P174968_LR_GCCF_on_Gowalla.ipynb)

![https://github.com/RecoHut-Stanzas/S794944/raw/main/images/process_flow.svg](https://github.com/RecoHut-Stanzas/S794944/raw/main/images/process_flow.svg)

## References

1. [https://github.com/RecoHut-Stanzas/S794944](https://github.com/RecoHut-Stanzas/S794944)
2. [https://arxiv.org/abs/2001.10167](https://arxiv.org/abs/2001.10167)
3. [https://github.com/newlei/LR-GCCF](https://github.com/newlei/LR-GCCF)